**SQL_3_HW_CHernov_Kirill**

1)
```sql
CREATE TABLE users (
    user_id SERIAL PRIMARY KEY,
    birth_date DATE NOT NULL,
    sex VARCHAR(6) NOT NULL CHECK(sex = 'male' OR sex = 'female'),
    age SMALLINT CHECK (age > 0)
);

-- Создание таблицы items
CREATE TABLE items (
    item_id SERIAL PRIMARY KEY,
    description VARCHAR(255) NOT NULL,
    price NUMERIC NOT NULL CHECK (price >= 0),
    category VARCHAR(50) NOT NULL
);

-- Создание таблицы ratings
CREATE TABLE ratings (
    item_id INT,
    user_id INT,
    review VARCHAR(255),
    rating SMALLINT CHECK (rating >= 0 AND rating <= 5),
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
```

Тип `SERIAL` используется для генерации последовательности целых чисел, которые используются в качестве первичного ключа таблиц  (поле user_id и item_id). Типы данных `DATE`, `VARCHAR` и `INT` используются для хранения дат, строк, и целых чисел соответственно. `NUMERIC` используется для хранения денежных сумм, где требуется точность. Ключевое слово `FOREIGN KEY` используется для создания внешних ключей, которые связывают таблицу ratings с таблицами items и users.

Генерация данных:
```sql

INSERT INTO users(birth_date, sex)
SELECT
       (DATE '1990-01-01' + INTERVAL '1 day' * random() * 7300) ::DATE AS birth_date,
       CASE WHEN random() < 0.5 THEN 'male' ELSE 'female' END AS sex,
       
FROM generate_series(1, 10);
```
Здесь генерируется дата рождения user'a случайным образом: к 1990-01-01 добавляется добавляется случайное количетсво дней до 20 лет (7300 дней), а также с вероятностью 0.5 пол.

```sql
UPDATE users
  SET age = EXTRACT(YEAR FROM age(birth_date));
```
Возраст вычисляется с использованием функции `age`, которая вычитает из текущей даты дату рождения.

Чтобы заполнить таблицу items воспользуемся "ручным методом" :) (ради интереса):
```sql
INSERT INTO items (description, price, category)
VALUES
    ('headphones', 10.95, 'electronics'),
    ('mouse', 4.50, 'electronics'),
    ('watch', 20.50, 'electronics'),
    ('laptop', 899.90, 'electronics'),
    ('lamp', 12.99, 'electronics'),
    ('pot', 8.40, 'dishes'),
    ('frying pan', 9.40, 'dishes'),
    ('plate red', 3.90, 'dishes'),
    ('plate green', 3.90, 'dishes'),
    ('plate yellow', 3.90, 'dishes'),
    ('fork', 2.75, 'dishes'),
    ('spoon', 2.75, 'dishes'),
    ('pen', 0.5, 'chancellery'),
    ('paper', 8.40, 'chancellery'),
    ('copybook', 0.90, 'chancellery'),
    ('pencil box', 1.34, 'chancellery'),
    ('compass', 3.45, 'chancellery'),
    ('milk', 1.40, 'products'),
    ('fish', 3.78, 'products'),
    ('meat', 3.04, 'products');
```

Генерация данных в ratings:

```sql
INSERT INTO ratings (item_id, user_id, review, rating)
SELECT
    items.item_id,
    users.user_id,
    CASE floor(random() * 5)
           WHEN 0 THEN 'Top!'
           WHEN 1 THEN 'Good'
           WHEN 2 THEN 'Nice'
           WHEN 3 THEN 'BAD!'
           WHEN 4 THEN 'Very BAD'
    END as review,
    FLOOR(RANDOM() * 5 + 1) as rating
FROM  users, items;
```



2) Связь/отношение между таблицами users и ratings - это один-к-многим (one-to-many). Одному пользователю может соответствовать несколько рейтингов.

Связь/отношение между таблицами items и ratings - также один-к-многим (one-to-many). Одному товару может соответствовать несколько рейтингов.

3)
Варианты создания индексов зависят от конкретных запросов и операций, которые будут выполняться на этих таблицах. В общем случае, рассмотрим несколько потенциальных вариантов:

1. Для таблицы users можно создать индекс по столбцу, который используется для идентификации пользователя, например, по полю "user_id". Это поможет ускорить запросы, которые ищут информацию о конкретном пользователе.

2. Для таблицы ratings можно создать индекс по полям, используемым в частых запросах. Например, если мы часто делаем выборку рейтингов по конкретному пользователю (user_id) и/или конкретному товару (item_id), то индексы по этим полям могут существенно ускорить такие запросы.

3. Для таблицы items можно создать индекс по полю, используемому для идентификации товара, например, по полю "item_id". Это будет полезно, если часто выполняются запросы на получение информации о конкретном товаре.

4)  
2 - может добавить в таблицу car_owner данные, потому что значение в первом столбце является уникальным (primary key), все типы соответствуют типам колонок.
7 - может добавить в таблицу car данные, потому что значение в первом столбце может наследоваться от primary key таблицы car_owner, все типы соответствуют типам колонок.


5) Да, данный скрипт можно использовать для шардинга таблицы на 32 документа и генерации 16 тестовых таблиц.